# Exploring RAG Workflow 

## Setup

First, we import the necessary modules from our library and initialize the `LLMManager`.

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:

from figma_llm.models.llm import LLMManager
from figma_llm.embeds.extract import EmbeddingModel
from figma_llm.embeds.db import EmbeddingStorage
from figma_llm.utils.config import Config



In [13]:


llm_manager = LLMManager(
    model_path=Config.MODEL_PATH,
    use_embeddings=True,
    embedding_model_info=Config.DEFAULT_EMBEDDINGS,
)
print("LLM Manager initialized.")




llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /Users/cck/repos/llama.cpp/models/Mistral-7B-Instruct-v0.2/mistral-7b-instruct-v0.2.Q4_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32        

LLM Manager initialized.


In [14]:

sample_documents = [
    "The capital of France is Paris.",
    "The Eiffel Tower is a famous landmark in Paris.",
    "France is known for its delicious cuisine, including croissants and baguettes.",
    "The Louvre Museum in Paris houses the famous painting, the Mona Lisa.",
]
for doc in sample_documents:
    llm_manager.embed_and_store(doc)
print("Sample documents embedded and stored.")




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Sample documents embedded and stored.


In [15]:
import os
file_path = os.path.join( f"{os.getcwd()}/../data/default_embeddings")
setattr(llm_manager.embedding_storage, "file_path", file_path)

In [16]:
llm_manager.embedding_storage._save_to_file()
print("Embeddings saved to file.")

llm_manager.embedding_storage._load_from_file()
print("Embeddings loaded from file.")



Embeddings saved to file.
Embeddings loaded from file.


In [17]:
query = "Tell me about a famous landmark in France."
query_embed = llm_manager.embedding_model.embed(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
top_similar = llm_manager.embedding_storage.find_top_n(query_embed, n=2)


In [19]:
top_similar

[('95ef45662a5470c5c533d0ffe4d6d4e59c9eef10da7afc42e7fce6873977eab6',
  0.3568437440386105,
  'The Eiffel Tower is a famous landmark in Paris.'),
 ('a1b7eb2ee7a6aded8dda4e6cf30826f5afffb28a5597ee9389e91eb326d4e319',
  0.37764341038584404,
  'The capital of France is Paris.')]

In [20]:

context = "\n".join([str(text) for _, _, text in top_similar])

response = llm_manager.generate_response(f"{query}\nContext: {context}")
print(f"Response: {response}")



llama_print_timings:        load time =    4602.63 ms
llama_print_timings:      sample time =      24.85 ms /   256 runs   (    0.10 ms per token, 10303.47 tokens per second)
llama_print_timings: prompt eval time =    4602.55 ms /    35 tokens (  131.50 ms per token,     7.60 tokens per second)
llama_print_timings:        eval time =   12670.45 ms /   255 runs   (   49.69 ms per token,    20.13 tokens per second)
llama_print_timings:       total time =   17644.35 ms /   290 tokens


Response:  One of its most iconic structures and a major tourist attraction is the Eiffel Tower. Named after the engineer Gustave Eiffel, who oversaw its construction, this wrought iron lattice tower was built between 1887 and 1889 as the entrance arch for the 1889 Exposition Universelle (World's Fair) held to celebrate the 100th year anniversary of the French Revolution. Standing at 1,063 feet (324 meters) tall, including its antenna, it was, at that time, the tallest man-made structure in the world.

The tower's four pillars are bent at a slight outward angle to counteract swaying caused by wind, while the first and third levels have restaurants. The second level has an observation deck, while the third level and a small observation deck called the Champ de Mars offer panoramic views of the city. It's estimated that over 67 million people have visited the tower since it opened to the public.

The Eiffel Tower was originally criticized by many artists and intellectuals for being an ey